# Planet: Understanding the Amazon from Space

## Make predictions and create submission file

In [1]:
from fastai.vision import *

In [2]:
path = Path('../data')
path

PosixPath('../data')

In [3]:
test = ImageList.from_folder(path/'test-jpg')
len(test)

61191

In [4]:
model = 'densenet121.pkl'
submission_fname = 'sub-densenet121-all-data-tta-thresh-17.csv'

I'll use [Test-Time Augmentation](https://machinelearningmastery.com/how-to-use-test-time-augmentation-to-improve-model-performance-for-image-classification/) to make predictions, which is now a built-in feature of the fastai library.

In [5]:
learn = load_learner(path, model, test=test)
preds, targets = learn.TTA(scale=1.05, ds_type=DatasetType.Test)

In [6]:
opt_thresh = False
if opt_thresh:
    # thresholds found after running get_thresholds.ipynb
    thresholds = [0.18,0.19,0.2,0.2,0.14,0.16,0.18,0.13,0.21,0.15,0.17,0.06,0.18,0.12,0.22,0.20,0.06 ]
    labelled_preds = [' '.join([learn.data.classes[i] for i,p in enumerate(pred) if p > thresholds[i]]) for pred in preds]
else:
    thresh = 0.17
    labelled_preds = [' '.join([learn.data.classes[i] for i,p in enumerate(pred) if p > thresh]) for pred in preds]

In [7]:
labelled_preds[:5]

['haze primary',
 'haze primary',
 'clear primary water',
 'clear primary',
 'agriculture clear cultivation habitation primary road']

In [8]:
fnames = [f.name[:-4] for f in learn.data.test_ds.items]

In [9]:
sub_df = pd.DataFrame({'image_name':fnames, 'tags':labelled_preds}, columns=['image_name', 'tags'])

In [10]:
sub_df.to_csv(path/'submissions'/submission_fname, index=False)

In [11]:
! kaggle competitions submit planet-understanding-the-amazon-from-space -f {path/'submissions'/submission_fname} -m "My submission"

100%|███████████████████████████████████████| 2.24M/2.24M [00:03<00:00, 656kB/s]
Successfully submitted to Planet: Understanding the Amazon from Space